# Business Understanding

Our MLB team wishes to improve upon our prior season's record in order to increase our chances of making a deep postseason run and winning the World Series next season.  Our offensive production was a weakness during the prior season.  We would like to use OPS to evaluate and predict the offensive production of MLB hitters.  OPS combines on-base skills (OBP or on-base percentage) with power hitting skills (slugging percentage), to measure overall offensive performace.  We will use this information to help build our roster for next season (evaluate our current under-contract players and possible trade acquisitions, as well as free agents).  Targeting hitters with a high OPS this offseason will help our team score more runs, win more games, perform better in the postseason, win a championship, and improve fan sentiment, driving revenue and profits in the process.

# Data Understanding

Our dataset is part of the Lahman Baseball database and obtained from Kaggle, compiled by author and journalist Sean Lahman.  It contains complete baseball statistics and data dating from 1871 to 2024.  We will use one of many tables, the "batting" table, for our purposes of evaluating and predicting offensive production.

In [158]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [159]:
#install Kaggle API to read data in directly from the site

!pip install kaggle

In [160]:
# define download path and copy the file

import os
import shutil

downloads_path = os.path.expanduser('~/Downloads/kaggle.json')
target_path = os.path.join(os.getcwd(), 'kaggle.json')

if os.path.exists(downloads_path):
    shutil.copy(downloads_path, target_path)
    print(f"kaggle.json copied to: {target_path}")
else:
    print("kaggle.json not found in Downloads.  Donwload it first lol!!!")

kaggle.json copied to: /Users/buzzardsroostimac/Documents/Flatiron/Phase_5/MLB_hitter_production/MLB_hitter_production/kaggle.json


In [161]:
# set permissions

!chmod 600 kaggle.json

In [162]:
!cp ~/Downloads/kaggle.json ~/.kaggle/kaggle.json

In [163]:
# providing API token credentials

!cat ~/.kaggle/kaggle.json

{"username":"shannonhunley","key":"e3f14a20e8743dcdf2de9f39646fcd05"}

In [164]:
# set permissions

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# downloading the Lahman baseball database which contains the hitting data

!kaggle datasets download -d dalyas/lahman-baseball-database

Dataset URL: https://www.kaggle.com/datasets/dalyas/lahman-baseball-database
License(s): CC-BY-SA-3.0


In [ ]:
# unzip the data

!unzip -o lahman-baseball-database.zip

In [ ]:
# reading in the data, getting a look at its columns and some values
df = pd.read_csv('lahman_1871-2024_csv/Batting.csv')

df.head()

In [ ]:
# looking at the amount of data and columns

df.info()

In [ ]:
# looking at the values of column 'G_old' since I'm unfamiliar with it, to see what info it provides

df['G_old'].value_counts()

The data contains plenty of records for modeling and includes all of the stats we need to compute our target, OPS.  We have a few null values to deal with and we have a few irrelevant columns we can remove.

## Data Preparation

In [ ]:
# dropping unnecessary columns

df_clean = df.drop(columns=['stint', 'teamID', 'lgID', 'G', 'G_batting', 'SB', 'CS', 'GIDP', 'G_old'])
df_clean.head()

In [ ]:
# confirming uneeded columns dropped and looking into null values

df_clean.info()

In [ ]:
# look at values for one of the columns containing null values

df_clean['RBI'].value_counts()

In [ ]:
# look at total nulls per column by number and percentage

print(df_clean.isnull().sum())

print((df_clean.isnull().sum() / len(df_clean)) * 100)

# look at some nulls in particular columns

df_clean[df_clean['HBP'].isnull()].head()

Certain columns contain null values because some of these stats were not tracked prior to certain dates.  It makes sense to fill in zero for nulls in count-based stats.  The oldest data is not going to factor heavily in our conclusions because we are looking for data relating to current players.

In [ ]:
# filling null values with 0

count_cols = ['RBI', 'SO', 'IBB', 'HBP', 'SH', 'SF']
df_clean[count_cols] = df_clean[count_cols].fillna(0)

Our goal is to build a roster using OPS as a measure of offensive performance using current players, so using current data is important.  Let's use 2000 as our cutoff year, as current players will have debuted by then and we still have a big enough dataset for robust modeling.  We will also limit our data by defining a qualified hitter as a player with 125 or more at bats to avoid basing our conclusions on the smallest sample sizes.

In [ ]:
# filter for modern baseball era
df_clean = df_clean[df_clean['yearID'] >= 2000]

# filter for qualified hitters
df_clean = df_clean[df_clean['AB'] >= 125]

print(df_clean.shape)

In [ ]:
# confirm the dataset size, columns, and that null values are removed
df_clean.info()

Now we have 10,000+ records for robust modeling, they are modern era baseball records and contain many current players we can build our team around and/or target in trades or free agency!

# Exploratory Data Analysis

Now we can calculate our target (OPS) and explore its relationship(s), if any, to our other data!  OPS is calculated as follows:
OPS = OBP + SLG
OBP is on-base percentage which measures how often a player gets on base (hits, walks, and hit by pitch).
SLG is slugging percentage, which measures a hitter's power by dividing total bases by total at-bats.

In [ ]:
# feature-engineering our target variable, OPS, and its components

# calculate OBP or on-base percentage, avoid throwing data with division by zero
numerator_obp = df_clean['H'] + df_clean['BB'] + df_clean['HBP']
denominator_obp = df_clean['AB'] + df_clean['BB'] + df_clean['HBP'] + df_clean['SF']
denominator_obp = denominator_obp.replace(0, np.nan) # avoid division by zero
df_clean['OBP'] = numerator_obp / denominator_obp

# calculate total bases
df_clean['TB'] = (df_clean['H'] - df_clean['2B'] - df_clean['3B'] - df_clean['HR']) + (2 * df_clean['2B']) + (3 * df_clean['3B']) + (4 * df_clean['HR'])

# calculate slugging percentage, avoid throwing data by dividing by zero
df_clean['SLG'] = df_clean['TB'] / df_clean['AB'].replace(0, np.nan) # avoid division by zero

# calculate OPS
df_clean['OPS'] = df_clean['OBP'] + df_clean['SLG']
df_clean['OPS'] = df_clean['OPS'].fillna(0)


In [ ]:
# preview target
print(df_clean[['playerID', 'yearID', 'OPS']].head(60))

Acuna showing most seasons with an .800+ OPS looks about right!  These values make sense.  Now we're going to drop the variable columns containing raw stats which which feed into the calculation of OPS to avoid multicollinearity, because we already know those relationships exist.  We keep other features like R, RBI, SO, etc. for modeling.

In [ ]:
# identify redundant columns
ops_component_cols = ['AB', 'H', '2B', '3B', 'HR', 'BB', 'HBP', 'SF', 'TB', 'OBP', 'SLG']

# drop redundant columns
df_clean = df_clean.drop(columns=ops_component_cols, errors='ignore')

# preview remaining data
df_clean.info()

Let's investigate some relationships between OPS and other remaining data.  Age is an important variable not contained in our data, so let's add that in from another CSV contained in the Lahman baseball database (People).

In [ ]:
# read in People.csv
people = pd.read_csv('lahman_1871-2024_csv/People.csv')

#calculate age
people['birthYear'] = people['birthYear'].fillna(0).astype(int)
df_clean = pd.merge(df_clean, people[['playerID', 'birthYear']], on='playerID', how='left')
df_clean['age'] = df_clean['yearID'] - df_clean['birthYear']
df_clean['age'] = df_clean['age'].clip(18, 50) # clip outliers

# drop birth year since we have age now
df_clean = df_clean.drop(columns=['birthyear'], errors='ignore')

# preview data containing age
df_clean.info()

In [ ]:
df_clean = df_clean.sort_values(['playerID', 'yearID'])
df_clean["OPS_next"] = df_clean.groupby('playerID')['OPS'].shift(-1)

df_train = df_clean[df_clean['yearID'] < 2024].dropna(subset=['OPS_next'])

df_2024 = df_clean[df_clean['yearID'] == 2024].copy

In [ ]:
numeric_cols = ['R', 'RBI', 'SO', 'IBB', 'SH', 'age', 'OPS', 'OPS_next']
corr_matrix = df_clean[numeric_cols].corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap of Key Features and OPS')
plt.show

We can see a notable lack of relationship between age and the other stats above, so let's investigate that further.

In [ ]:
# plot age vs. OPS

sns.set_style('whitegrid')
sns.set_palette('deep')

plt.figure(figsize=(12,7))
sns.regplot(
    x='age'
    y='OPS'
    data=df_clean
    scatter_kws={'alpha': 0.4, 'color': 'skyblue', 's': 20}
    line_kws={'color': 'navy', 'lw': 2},
    lowess=True
)

age_means = df_clean.groupby('age')['OPS'].mean().reset_index()
sns.scatterplot(
    x='age'
    y='OPS'
    data=age_means,
    color='orange',
    s=100,
    label='Mean OPS per Age'
)

plt.title('MLB Player Age vs. OPS')
plt.xlabel('Player Age')
plt.ylabel('OPS', fontsize=14)

plt.xlim(18,45)

peak_age = age_means.loc[age_means['OPS'].idxmax(), 'age']
peak_OPS = age_means['OPS'].max()
plt.annotate(
    f'Peak at Age {int(peak_age)} (OPS: {peak_OPS:.3f})',
    xy=(peak_age, peak_ops),
    xytext=(peak_age + 2, peak_ops - 0.05),
    arrowprops=dict(facecolor='black', arrowstyle='->'),
    fontsize=12
)

plt.legend(fontsize=12)
plt.tight_layout()

plt.show

# Modeling

# Evaluation

# Conclusion

## Limitations

## Recommendations

## Next Steps